# Domain Definiftions

In [162]:
from collections import deque as stack
from dataclasses import dataclass
from enum import Enum
from typing import List, Set, Dict, Tuple, Optional


In [171]:
@dataclass(frozen=False)
class Node:
    x: int
    y: int
    blocked: bool

    def __init__(self, x: int, y: int, blocked: bool = Node.is_blocked()):
        self.x = x
        self.y = y
        self.blocked = False

    """
    Generate a binomial random distribution of '0' and '1's
    where '0's represent the node is blocked, and therefore 
    it cannot be reached
    """
    @classmethod
    def is_blocked(cls):
        dist = np.random.binomial(1, 0.8)
        return True if dist == 0 else False

    @property
    def id(self) -> Tuple[int, int]:
        return self.x, self.y

    def is_free(self):
        return not self.blocked

    def find_neighbors(self):
        pass

    def __hash__(self):
        return self.id.__hash__()

In [172]:
class Terrain(Enum):
    Asphalt = ("Terra", 1)
    Flooding = ("Agua", 3)
    Quicksand = ("Areia Movedica", 6)


In [173]:
@dataclass(frozen=False)
class Edge:
    source: Node
    destination: Node
    terrain: Terrain

    def __init__(self, source: Node, dest: Node, terrain: Terrain = Terrain.Asphalt):
        self.source = source
        self.destination = dest
        self.terrain = terrain

    @property
    def weight(self):
        return self.terrain.value[1]
    
    def update_terrain(self, terrain: Terrain):
        self.terrain = terrain

In [174]:
@dataclass(frozen=False)
class Graph:
    nodes: Dict[Tuple[int,int], Node]
    edges: Dict[Tuple[Node,Node], Edge]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()

    def find_neighbors_of(self, node: Node):
        if node.is_free():
            for pos in self._find_neighbors_pos_estimation(node):
                neighbor: Optional[Node] = self.nodes.get(pos)
                if neighbor != None and neighbor.is_free():
                    yield self.nodes.get(pos)

    def find_edge_connecting(self, source: Node, dest: Node):
        return self.edges.get((source, dest))

    """
    estimated positions return a list of expected neighbors as follows: 
    up, down, left, right, up_right, up_left, down_right, down_left
    """
    def _find_neighbors_pos_estimation(self, node: Node):
        pos = node.id
        estimated_neighbors = [
            (pos[0], pos[1]-1),
            (pos[0], pos[1]+1),
            (pos[0]-1, pos[1]),
            (pos[0]+1, pos[1]),
            (pos[0]+1, pos[1]-1),
            (pos[0]-1, pos[1]-1),
            (pos[0]+1, pos[1]+1),
            (pos[0]-1, pos[1]+1)
        ]
        return estimated_neighbors

    def with_node(self, node: Node):
        self.nodes[node.id] = node
        return self

    def with_nodes(self, nodes: List[Node]):
        [self.with_node(node) for node in nodes]
        return self

    def with_edge(self, edge: Edge):
        src = edge.source
        dest = edge.destination
        self.edges[src,dest] = edge
        return self

    def with_edges(self, edges: List[Edge]):
        [self.with_edge(edge) for edge in edges]
        return self

    def reset(self):
        return Graph()


# Graph Construction

### Building the graph

In [270]:
import numpy as np
import matplotlib.pyplot as pt
import math

In [271]:
def generate_nodes(width: int, height: int):
    for row_num in range(width):
        for col_num in range(height):
            yield Node(row_num, col_num)

In [272]:
area_width = 3
area_height = 4

In [273]:
nodes = list(generate_nodes(area_width, area_height))
graph = Graph().with_nodes(nodes)

In [279]:
## For debugging purposes only

entries = [(1,1), (1,2), (2,2)]

for entry in entries:
    node = graph.nodes.get(entry)
    node.blocked = True

In [280]:
for node in graph.nodes.values():
    neighbors: List[Node] = graph.find_neighbors_of(node)
    edges: List[Edge] = [Edge(node, neighbor) for neighbor in neighbors]
    graph.with_edges(edges)

### Sort Terrains in Graph to screw with the edges originating from it

# PathFinder Algorithm

### A* Implementation

In [282]:
class AStarAlgorithm:
    graph: Graph
    visited_nodes: Set[Node]

    def __init__(self, graph: Graph):
        self.graph = graph
        self.visited_nodes = set()
        self.path_stack = stack()

    def shortest_path_between(self, source: Node, target: Node) -> List[Node]:
        neighbors: Set[Node] = set(graph.find_neighbors_of(source))
        neighbors = neighbors.difference(visited_nodes)

        if len(neighbors) > 0:
            edges: List[Edge] = [graph.find_edge_connecting(source, neighbor) for neighbor in neighbors]
            nodes_heuristics = [a_star.distance_between(edge.destination, target, edge.weight) for edge in edges]
            best_node_heuristic = min(nodes_heuristics, key=lambda h: h[1])
            selected_neighbor = best_node_heuristic[0]

            self.visited_node.add(source)
            self.path_stack.append(source)
            shortest_path_between(source=selected_neighbor, target=target)

        else:
            # Rollback
            last_visited_node = self.path_stack.pop()
            visited_nodes.remove(last_visited_node)
            visited.nodes.add(source)



    def distance_between(self, node: Node, target: Node, cost_to_node: int):
        euclidian_dist = pow(target.x - node.x,2) + pow(target.y - node.y,2)
        heuristic = math.sqrt(euclidian_dist) + cost_to_node
        return node, heuristic

In [110]:
a_star = AStarAlgorithm(graph)
source = graph.nodes.get((0,0))
dest = graph.nodes.get((area_width-1, area_height-1))

In [133]:
set(graph.find_neighbors_of(Node(0,0)))

{Node(x=0, y=1, blocked=False),
 Node(x=1, y=0, blocked=False),
 Node(x=1, y=1, blocked=False)}

In [65]:
possibilities

{'selected': Node(x=1, y=1, blocked=False),
 'neighbors': [Node(x=0, y=1, blocked=False),
  Node(x=1, y=0, blocked=False),
  Node(x=1, y=1, blocked=False)]}

# Graph Visualization

In [88]:
x = Node(0, 0)
y = Node(0, 5)

In [89]:
a = set()

In [90]:
a.add(x)
a.add(x)

In [91]:
a

{Node(x=0, y=0, blocked=False)}

In [113]:
stack.append(1)

TypeError: descriptor 'append' for 'collections.deque' objects doesn't apply to a 'int' object

In [96]:
from collections import deque

In [114]:
s = stack()

In [115]:
s.append(x)

In [116]:
s


deque([Node(x=0, y=0, blocked=False)])